In [50]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import duckdb
from bertopic.representation import  MaximalMarginalRelevance, OpenAI
import openai
import os

In [48]:
file_pages = '/workspace/eval_result_epic.json'
openai_key = os.getenv('OPENAI_API_KEY')

In [49]:
client = openai.OpenAI(api_key=openai_key)

In [58]:
embeding_model = SentenceTransformer('all-MiniLM-L6-v2')
mmr = MaximalMarginalRelevance(diversity=0.3)
openai_generator = OpenAI(client, model="gpt-4o-mini", chat=True)
representation_models = [mmr, openai_generator]

In [71]:
kb_pages =  duckdb.sql(f"SELECT contexts FROM '{file_pages}'")
docs = [d[0][0] for d in kb_pages.fetchall()]
len(docs)

194

In [76]:
topic_model = BERTopic(representation_model=representation_models, nr_topics=20, verbose=True)
topics, probs = topic_model.fit_transform(docs)

2024-12-22 23:06:31,621 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 7/7 [00:05<00:00,  1.31it/s]
2024-12-22 23:06:37,693 - BERTopic - Embedding - Completed ✓
2024-12-22 23:06:37,693 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-12-22 23:06:37,934 - BERTopic - Dimensionality - Completed ✓
2024-12-22 23:06:37,935 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-12-22 23:06:37,938 - BERTopic - Cluster - Completed ✓
2024-12-22 23:06:37,938 - BERTopic - Representation - Extracting topics from clusters using representation models.
100%|██████████| 9/9 [00:05<00:00,  1.74it/s]
2024-12-22 23:06:44,066 - BERTopic - Representation - Completed ✓
2024-12-22 23:06:44,068 - BERTopic - Topic reduction - Reducing number of topics
2024-12-22 23:06:44,071 - BERTopic - Topic reduction - Reduced number of topics from 9 to 9


In [ ]:
topic_model.get_topic_info()['Representation']

0    [Zoom Meeting Guidelines and Waiting Room Proc...
1           [Okta Verify Login Issues and Resolutions]
2    [DNS and GlobalProtect VPN Troubleshooting Guide]
3             [Notion Workspace Management and Macros]
4                  [Jira Access Requests in SailPoint]
5               [Epic Connect and Jira Upgrade Issues]
6    [Password Management and Multi-Factor Authenti...
7    [Access Request Process for Adobe Creative Clo...
8    [Access Request Instructions for CLEAR and Syn...
Name: Representation, dtype: object


In [43]:
topic_model.get_document_info(docs)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,## Issue\n\n * My phone is not showing GMail...,0,0_page_state_click_add,"[page, state, click, add, label, archive, goog...","[To archive, add the label ` archive ` to the ...",page - state - click - add - label - archive -...,1.000000,False
1,## Issue\n\n * Can I get access to CLEAR? \n...,1,1_click_access_application_sailpoint,"[click, access, application, sailpoint, finish...",[## Issue\n\n * Can I get access to CLEAR? \...,click - access - application - sailpoint - fin...,1.000000,True
2,## Issue\n\n * I need access to VNDLY \n\n##...,3,3_sailpoint_center_search_creative,"[sailpoint, center, search, creative, applicat...",[## Issue\n\n * I need access to Adobe Creat...,sailpoint - center - search - creative - appli...,0.961107,True
3,## Issue\n\n * Can't find user in VNDLY to d...,5,5_zoom_room_participants_join,"[zoom, room, participants, join, external, con...","[If you would like to create a persistent, ope...",zoom - room - participants - join - external -...,1.000000,False
4,## Issue\n\n * I need access to Monotype Fon...,3,3_sailpoint_center_search_creative,"[sailpoint, center, search, creative, applicat...",[## Issue\n\n * I need access to Adobe Creat...,sailpoint - center - search - creative - appli...,0.964821,False
...,...,...,...,...,...,...,...,...
189,### Issue\n\n * I need access to SyncSketch ...,1,1_click_access_application_sailpoint,"[click, access, application, sailpoint, finish...",[## Issue\n\n * Can I get access to CLEAR? \...,click - access - application - sailpoint - fin...,1.000000,True
190,## Issue\n\n * I need access to Corporate Tr...,3,3_sailpoint_center_search_creative,"[sailpoint, center, search, creative, applicat...",[## Issue\n\n * I need access to Adobe Creat...,sailpoint - center - search - creative - appli...,1.000000,False
191,## Issue\n\n * I need envelope access on Doc...,1,1_click_access_application_sailpoint,"[click, access, application, sailpoint, finish...",[## Issue\n\n * Can I get access to CLEAR? \...,click - access - application - sailpoint - fin...,0.721763,False
192,"To archive, add the label ` archive ` to the p...",0,0_page_state_click_add,"[page, state, click, add, label, archive, goog...","[To archive, add the label ` archive ` to the ...",page - state - click - add - label - archive -...,1.000000,False
